In [1]:
uk2gtfs_version <- "f15694a655c508f8caebaf99328b0a2d1bc8dfa5"

if (!require("pacman")) install.packages("pacman")
pacman::p_load(remotes, rnaturalearth, sf)

if (!require(UK2GTFS)) { remotes::install_github("ITSleeds/UK2GTFS", ref = uk2gtfs_version)}

library(UK2GTFS)

Loading required package: pacman
Loading required package: UK2GTFS
Your UK2GTFS data is up to date


In [20]:
out_dir <- "./out"
out_name <- "rail_scot_gtfs"
preview_mode <- TRUE

timetable_zip <- file.path(out_dir, "timetable.zip")
timetable_dir <- file.path(out_dir, "timetable")

dir.create(out_dir, recursive = TRUE, showWarnings = FALSE)

In [2]:
scotland_highres <- rnaturalearth::ne_download(
  scale = 10L,
  type = "map_subunits",
  category = "cultural",
  returnclass = "sf"
) |> subset(SU_A3=="SCT")

scotland <- scotland_highres |>
  sf::st_buffer(1000)

Reading ]8;;file:///Users/daniel/projects/munro-access/ne_10m_admin_0_map_subunits.zipne_10m_admin_0_map_subunits.zip]8;; from naturalearth...


In [4]:
if (dir.exists(timetable_dir) && length(list.files(timetable_dir)) > 0) {
  print(paste("Using existing timetable from", timetable_dir))
} else {
  print(paste("Downloading timetable from NRDP to", timetable_zip, "..."))
  nrdp_timetable(timetable_zip)

  print(paste("Unzipping timetable to", timetable_dir, "..."))
  dir.create(timetable_dir)
  unzip(timetable_zip, exdir = timetable_dir)
}

[1] "Using existing timetable from ./out/timetable"


In [5]:
print("atoc2gtfs...")
gtfs_raw <- atoc2gtfs(
  path_in = "out/timetable.zip",
  ncores = 4
)

[1] "atoc2gtfs..."
Adding 69 missing tiplocs, these may have unreliable location data
2025-12-09 22:28:38.864976 Some calendar dates had incorrect start or end dates that did not align with operating day bitmask.
 Services=G37699,C25987,C27192,L36706,L36756,L36896,L36952,L11878,L11845,L11888
2025-12-09 22:28:39.26426 Constructing calendar and calendar_dates


 Progress: ───────────────────────────────────────────────────────────────  100%Progress: ─────────────────────────────────────────────────────────        100%Progress: ─────────────────────────────────────────────────────────        100%Progress: ───────────────────────────────────────────────────────────      100%Progress: ───────────────────────────────────────────────────────────      100%Progress: ───────────────────────────────────────────────────────────      100%Progress: ───────────────────────────────────────────────────────────      100%Progress: ───────────────────────────────────────────────────────────      100%Progress: ────────────────────────────────────────────────────────────     100%Progress: ────────────────────────────────────────────────────────────     100%Progress: ─────────────────────────────────────────────────────────────    100%Progress: ─────────────────────────────────────────────────────────────    100%Warning message:
In readLines(con = file, n = -1) :

In [11]:
print("Clipping to Scotland...")
gtfs <- gtfs_clip(gtfs_raw, scotland)

[1] "Clipping to Scotland..."


In [34]:
print("Validate (raw Scotland data):")
gtfs_validate_internal(gtfs)

print("Cleaning...")

gtfs <- gtfs_clean(gtfs, public_only=TRUE)

if (!is.null(gtfs$transfers)) {
  gtfs$transfers <- gtfs$transfers[
    gtfs$transfers$from_stop_id %in% gtfs$stops$stop_id &
    gtfs$transfers$to_stop_id %in% gtfs$stops$stop_id,
  ]
}

out_of_order_times <- gtfs$stop_times$arrival_time > gtfs$stop_times$departure_time
if (any(out_of_order_times)) {
  message(sprintf("Swapping %d stop_times with arrival > departure", sum(out_of_order_times)))
  temp <- gtfs$stop_times$arrival_time[out_of_order_times]
  gtfs$stop_times$arrival_time[out_of_order_times] <- gtfs$stop_times$departure_time[out_of_order_times]
  gtfs$stop_times$departure_time[out_of_order_times] <- temp
}

print("Validate (Cleaned):")
gtfs_validate_internal(gtfs)

print("Forcing valid...")
gtfs <- gtfs_force_valid(gtfs)

print("Validate (Final):")
gtfs_validate_internal(gtfs)

# gtfs <- gtfs_compress(gtfs)

if (preview_mode) {
  gtfs <- gtfs_trim_dates(
    gtfs,
    startdate = 20251210L,
    enddate = 20251220L
  )
}

[1] "Validate (raw Scotland data):"
train_category are invalid columns in routes.txt
power_type are invalid columns in trips.txt
NA values in trips
[1] "Cleaning..."
Swapping 85 stop_times with arrival > departure
[1] "Validate (Cleaned):"
train_category are invalid columns in routes.txt
power_type are invalid columns in trips.txt
NA values in trips
[1] "Forcing valid..."
This function does not fix problems it just removes them
[1] "Validate (Final):"
train_category are invalid columns in routes.txt
power_type are invalid columns in trips.txt
NA values in trips
Trimming GTFS between 20251210 and 20251220


In [35]:
out_zip_path <- file.path(out_dir, paste(out_name, ".zip", sep=""))
unzipped_path <- file.path(out_dir, out_name)

print(paste("Writing", out_zip_path, "..."))
gtfs_write(gtfs,
  folder="./out",
  name=out_name # gtfs_write takes without .zip
)

dir.create(unzipped_path, showWarnings = FALSE)
unzip(out_zip_path, exdir=unzipped_path)

print("All done.")

[1] "Writing ./out/rail_scot_gtfs.zip ..."
[1] "All done."
